In [ ]:
import base64, sys, os, shutil, subprocess
import io

import numpy
import requests

requests.packages.urllib3.disable_warnings()

ATOMIC_SYMBOLS = {
    1: 'H', 2: 'He', 3: 'Li', 4: 'Be', 5: 'B', 6: 'C', 7: 'N', 8: 'O', 9: 'F', 10: 'Ne',
    # Add more elements as needed
}


### access to database
class CIDHelper:

	SERVER = 'https://69.12.4.38/CCSD'
	CIDs = []


	def ProcessResponse(rsp):

		if rsp.status_code != 200:
			print('unable to connect to server')
			return None

		answer = rsp.json()
		if answer['type'] == 'error':
			print('error response')
			return None

		return answer

	def GetCIDs():


		req = requests.get('{}/CIDs'.format(CIDHelper.SERVER), verify=False)
		answer = CIDHelper.ProcessResponse(req)
		if answer is None: return None

		CIDHelper.CIDs = answer['CIDs']


	def Init():
		CIDHelper.GetCIDs()


	def GetMolecule(CID):

		if CID not in CIDHelper.CIDs:
			print('molecule {} not available'.format(CID))
			return None


		req = requests.get('{}/dccsd/{}'.format(CIDHelper.SERVER, CID), verify=False)
		answer = CIDHelper.ProcessResponse(req)
		if answer is None: return None


		#answer['xyz'] = numpy.array(answer['xyz'])

		b = answer['D-CCSD'].get('$binary',{}).get('base64',None)
		if b is None:
			print("invalid molecule data")
			return None

		b = base64.b64decode(b)
		bio = io.BytesIO(b)

		matrix = numpy.load(bio)
		
		answer['CID'] = CID
		answer['D-CCSD'] = matrix
		return answer

	def GetXYZ(CID):

		if CID not in CIDHelper.CIDs:
			print('molecule {} not available'.format(CID))
			return None


		req = requests.get('{}/molecule/xyz/{}'.format(CIDHelper.SERVER, CID), verify=False)
		answer = CIDHelper.ProcessResponse(req)
		if answer is None: return None

		
		answer['CID'] = CID
		answer['Z'] = numpy.asarray(answer['Z'])
		answer['xyz'] = numpy.asarray(answer['xyz'])
		return answer


	def GetRandomMolecule():

		cid = numpy.random.choice(CIDHelper.CIDs)
		return CIDHelper.GetMolecule(cid)



def PrintMoleculeGaussian(cid_num,molecule_data):
    #if molecule_data is None:
    #    print("No molecule data to print")
    #    return

    #print("Molecule data keys:", molecule_data.keys())

    # Attempt to get atomic numbers and coordinates
    atomic_numbers = molecule_data.get('Z')
    coordinates = molecule_data.get('xyz')

    #if atomic_numbers is None or coordinates is None:
    #    print("Molecule data does not contain 'Z' or 'xyz'")
    #return

    num_atoms = len(atomic_numbers)

    # Print Gaussian header
    print("%nproc=32")
    print("%chk={}.chk".format(cid_num))
# PBEPBE/6-311G NoSymm freq=raman

#Title: Created by Jmol version 14.6.4_2016.11.05  2016-11-10 03:30


    print("# PBEPBE/6-31G(d) NoSymm freq=raman\n")
    print("This is molecule {}\n".format(cid_num))
    print("0 1")  # Charge and multiplicity, adjust as needed

    # Print atomic symbols and coordinates
    for atom_number, coord in zip(atomic_numbers, coordinates):
        atom_symbol = ATOMIC_SYMBOLS.get(atom_number, str(atom_number))
        x, y, z = coord
        print(f"{atom_symbol} {x:.6f} {y:.6f} {z:.6f}")

    print("")



CIDHelper.Init()

#specific_cid = 85310
#specific_molecule = CIDHelper.GetXYZ(specific_cid)
#if specific_molecule:
#    print(specific_molecule)
#
#specific_cid = 12283
#specific_molecule = CIDHelper.GetXYZ(specific_cid)
#if specific_molecule:
#    print(specific_molecule)



#specific_cid = 84310
#specific_cid = CIDHelper.GetCIDs()
#if specific_cid:
#    print(specific_cid)

#print('CIDs:', CIDHelper.CIDs)

    #cid=12283
for cid in CIDHelper.CIDs:
    specific_molecule = CIDHelper.GetXYZ(cid)
    atomic_numbers = specific_molecule.get('Z')

    if all(num < 9 for num in atomic_numbers):
        print('CID:', cid)
        with open(f"{cid}.com", "w") as f:
            sys.stdout = f
            PrintMoleculeGaussian(cid,specific_molecule)
            sys.stdout = sys.__stdout__
        #os.system('srun g16 <{}.com> {}.log'.format(cid, cid))
        #os.system('formchk {}.chk'.format(cid))


#random_molecule = CIDHelper.GetRandomMolecule()
#if random_molecule:
#    print(random_molecule)



# Fetch and print a specific molecule in Gaussian input format
        #specific_cid = 12283
        #specific_molecule = CIDHelper.GetXYZ(cid)